In [1]:
%pylab inline
import platform
import IPython
import sklearn as sk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

print ('Python version:', platform.python_version())
print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('scikit-learn version:', sk.__version__)
print ('matplotlib version:', matplotlib.__version__)

Populating the interactive namespace from numpy and matplotlib
Python version: 3.5.2
IPython version: 4.0.1
numpy version: 1.13.1
scikit-learn version: 0.18.2
matplotlib version: 1.5.0


In the previous chapters we have studied several algorithms for very different tasks,
from classification and regression to clustering and dimensionality reduction. We
showed how we can apply these algorithms to predict results when faced with new
data. That is what machine learning is all about. In this last chapter, we want to show
some important concepts and methods you should take into account if you want to
do real-world machine learning.
- In real-world problems, usually data is not already expressed by attribute/
float value pairs, but through more complex structures or is not structured at
all. We will learn __feature extraction__ techniques that will allow us to extract
scikit-learn features from data.
- From the initial set of available features, not all of them will be useful
for our algorithms to learn from; in fact, some of them may degrade our
performance. We will address the problem of selecting the most adequate
feature set, a process known as __feature selection__.
- Finally, as we have seen in the examples in this book, many of the machine
learning algorithms have parameters that must be set in order to use them.
To do that, we will review __model selection__ techniques; that is, methods to
select the most promising hyperparameters to our algorithms. 

In [2]:
a = 5